# Version Benchmarking: HEC-RAS 6.1 to 6.6

## Overview

This notebook demonstrates **HEC-RAS version benchmarking** to quantify computational performance improvements across major releases. Understanding version-specific performance is essential for:

- **Upgrade Planning**: Evaluating whether version upgrades justify migration effort
- **Performance Regression**: Detecting if newer versions introduce slowdowns
- **Feature vs Performance Trade-offs**: Understanding computational cost of new capabilities
- **Reproducibility**: Documenting version-specific timing for project records

### What This Notebook Does

1. Extracts a 2D example project (BaldEagleCrkMulti2D)
2. Runs the same plan across multiple HEC-RAS versions (6.1 through 6.6)
3. Measures execution time for each version
4. Visualizes performance evolution across releases
5. Identifies the fastest and most stable version for this model

### LLM Forward Principle: Professional Responsibility

HEC-RAS version selection involves professional engineering judgment:
- **Regulatory Acceptance**: Some jurisdictions require specific versions
- **Feature Requirements**: Newer versions may be necessary for certain capabilities
- **Stability vs Performance**: Balance computational speed with proven reliability
- **Support Lifecycle**: Consider vendor support timelines

### HEC-RAS Version History Context

**HEC-RAS 6.x Series Major Changes**:
- **6.0 (2021)**: Initial 2D diffusion wave release
- **6.1 (2021)**: 2D stability improvements
- **6.2 (2022)**: Parallel processing enhancements
- **6.3 (2022)**: Terrain preprocessing optimization
- **6.4 (2023)**: 2D shallow water equations (full momentum)
- **6.5 (2024)**: GPU acceleration (beta), improved solver
- **6.6 (2024)**: Production GPU support, performance tuning

**Expected Trends**:
- **6.1 to 6.3**: Gradual performance improvements (10-20% faster)
- **6.4+**: Variable performance depending on equation set
- **6.5+**: Potential GPU speedup if model is GPU-compatible

### Reference

For HEC-RAS version release notes:
- [HEC-RAS Release Notes Archive](https://www.hec.usace.army.mil/software/hec-ras/documentation.aspx)
- [HEC-RAS Version Comparison Chart](https://www.hec.usace.army.mil/confluence/rasdocs/)

For version selection guidance:
- [HEC-RAS Best Practices: Version Selection](https://www.hec.usace.army.mil/confluence/rasdocs/)

### Important Considerations

**Before Running This Notebook**:
1. **Multiple Installations Required**: Each HEC-RAS version must be installed separately
2. **Installation Paths**: Verify paths in Parameters cell match your installations
3. **Licensing**: Ensure all versions are properly licensed
4. **Disk Space**: Allow ~2 GB per version for installations
5. **Execution Time**: Benchmarking 6 versions may take hours for large models


In [1]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
USE_LOCAL_SOURCE = False  # <-- TOGGLE THIS

if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)
    if local_path not in sys.path:
        sys.path.insert(0, local_path)
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")

# Import ras-commander
from ras_commander import HdfResultsPlan, RasCmdr, RasExamples, RasGeo, RasPlan, init_ras_project, ras

# Verify which version loaded
import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

📦 PIP PACKAGE MODE: Loading installed ras-commander
✓ Loaded: C:\Users\bill\AppData\Roaming\Python\Python313\site-packages\ras_commander\__init__.py


## Parameters

Configure these values to customize the notebook for your project.

## Expected Results and Interpretation

### Performance Metrics

This analysis will generate:

1. **Absolute Execution Time**: Wall-clock time for each version
2. **Relative Performance**: Normalized to baseline (6.1 = 100%)
3. **Version Trends**: Performance trajectory across releases

### Typical Patterns

**Incremental Releases** (6.1 to 6.3):
- 10-20% cumulative speedup
- Consistent performance (no regressions)

**Major Feature Additions** (6.4: Full Momentum Equations):
- Potential slowdown if using new features (more complex physics)
- Similar or faster performance if using legacy diffusion wave

**GPU Acceleration** (6.5+):
- 2-10x speedup for GPU-compatible models (large 2D meshes)
- Similar performance for small models or non-GPU runs

### Audit Trail Recommendations

Document for engineering records:
- **Hardware Configuration**: CPU, RAM, GPU (if applicable)
- **Model Specifications**: Mesh cells, equations used, simulation duration
- **Version Execution Times**: Tabular results with statistical summary
- **Version Selection Rationale**: Why specific version chosen for production
- **QA/QC Results**: Verification that results match across versions


In [2]:
# =============================================================================
# PARAMETERS - Edit these to customize the notebook
# =============================================================================
from pathlib import Path

# Project Configuration
PROJECT_NAME = "BaldEagleCrkMulti2D"  # Example project to extract
RAS_VERSION = "6.6"                   # Default HEC-RAS version for initial setup

# Version Benchmarking Settings
VERSIONS_TO_TEST = ['6.6', '6.5', '6.4.1', '6.3.1', '6.3', '6.2', "6.1", "6.0"]
PLAN = "02"                           # Plan number to benchmark across versions
NUM_CORES = 4                         # Cores per simulation

# All Plans Benchmark Settings
BENCHMARK_ALL_PLANS = True            # Whether to benchmark all plans
ALL_PLANS_VERSION = "6.6"             # Version for all-plans benchmark
ALL_PLANS_CORES = 2                   # Cores for all-plans benchmark

In [3]:
# =============================================================================
# DEPENDENCY CHECK AND AUTO-INSTALL
# =============================================================================
# Check for py-cpuinfo and install if missing
import subprocess
import sys

try:
    import cpuinfo
    print("✓ py-cpuinfo already installed")
except ImportError:
    print("⚠️ py-cpuinfo not found - installing...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "py-cpuinfo"])
        print("✓ Successfully installed py-cpuinfo")
        print("📝 Note: You may need to restart the kernel for the import to work")
        print("   Use: Kernel → Restart & Run All")
    except Exception as e:
        print(f"❌ Failed to install py-cpuinfo: {e}")
        print("   Manual installation: pip install py-cpuinfo")

✓ py-cpuinfo already installed


In [4]:
import h5py
import numpy as np
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
from shapely.geometry import Point, LineString, Polygon
import xarray as xr
from pathlib import Path
import sys
from concurrent.futures import ThreadPoolExecutor, as_completed
import psutil
import platform
import cpuinfo

In [ ]:
# Define versions to compare (from Parameters cell)
versions = VERSIONS_TO_TEST
versions


In [6]:
# Extract project using RasExamples - simple and reliable
project_path = RasExamples.extract_project(PROJECT_NAME, suffix="701")
print(f"✓ Using project: {project_path}")

2026-01-19 16:36:54 - ras_commander.RasExamples - INFO - Found zip file: C:\Users\bill\AppData\Local\ras-commander\examples\Example_Projects_6_6.zip
2026-01-19 16:36:54 - ras_commander.RasExamples - INFO - Loading project data from CSV...
2026-01-19 16:36:54 - ras_commander.RasExamples - INFO - Loaded 68 projects from CSV.
2026-01-19 16:36:54 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----
2026-01-19 16:36:54 - ras_commander.RasExamples - INFO - Extracting project 'BaldEagleCrkMulti2D' as 'BaldEagleCrkMulti2D_701'
2026-01-19 16:36:56 - ras_commander.RasExamples - INFO - Successfully extracted project 'BaldEagleCrkMulti2D' to c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_701


✓ Using project: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_701


In [7]:
# Init the ras_project with ras-commander to read all HEC-RAS project information 
init_ras_project(project_path, RAS_VERSION)
print(ras)
# If no ras object is defined in init_ras_project, it defaults to "ras" (useful for single project scripts)
# Display plan dataframe
ras.plan_df

2026-01-19 16:36:56 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.g06
2026-01-19 16:36:56 - ras_commander.geom.GeomLateral - INFO - Found 0 SA/2D connections in BaldEagleDamBrk.g06
2026-01-19 16:36:56 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.g08
2026-01-19 16:36:56 - ras_commander.geom.GeomLateral - INFO - Found 0 SA/2D connections in BaldEagleDamBrk.g08
2026-01-19 16:36:56 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.g10
2026-01-19 16:36:56 - ras_commander.geom.GeomLateral - INFO - Found 0 SA/2D connections in BaldEagleDamBrk.g10
2026-01-19 16:36:56 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.g11
2026-01-19 16:36:56 - ras_commander.geom.GeomLateral - INFO - Found 0 SA/2D connections in BaldEagleDamBrk.g11
2026-01-19 16:36:56 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.

,plan_number,unsteady_number,geometry_number,Plan Title,Program Version,Short Identifier,Simulation Date,Computation Interval,Mapping Interval,Run HTab,...,Friction Slope Method,UNET D2 SolverType,UNET D2 Name,HDF_Results_Path,Geom File,Geom Path,Flow File,Flow Path,full_path,flow_type
0,13,07,06,PMF with Multi 2D Areas,5.10,PMF Multi 2D,"01JAN1999,1200,04JAN1999,1200",30SEC,30MIN,1,...,1,Pardiso (Direct),193,None,06,C:\GH\ras-commander\examples\example_projects\...,07,C:\GH\ras-commander\examples\example_projects\...,C:\GH\ras-commander\examples\example_projects\...,Unsteady
1,15,12,08,1d-2D Dambreak Refined Grid,5.10,1D-2D Refined Grid,"01JAN1999,1200,04JAN1999,1200",20SEC,5MIN,1,...,1,NaN,BaldEagleCr,None,08,C:\GH\ras-commander\examples\example_projects\...,12,C:\GH\ras-commander\examples\example_projects\...,C:\GH\ras-commander\examples\example_projects\...,Unsteady
2,17,09,10,2D to 1D No Dam,5.00,2D to 1D No Dam,"01JAN1999,1200,06JAN1999,1200",1MIN,5MIN,1,...,1,NaN,Upstream2D,None,10,C:\GH\ras-commander\examples\example_projects\...,09,C:\GH\ras-commander\examples\example_projects\...,C:\GH\ras-commander\examples\example_projects\...,Unsteady
3,18,10,11,2D to 2D Run,5.00,2D to 2D Run,"01JAN1999,1200,04JAN1999,1200",20SEC,5MIN,1,...,1,NaN,BaldEagleCr,None,11,C:\GH\ras-commander\examples\example_projects\...,10,C:\GH\ras-commander\examples\example_projects\...,C:\GH\ras-commander\examples\example_projects\...,Unsteady
4,19,11,12,SA to 2D Dam Break Run,5.00,SA to 2D Dam Break,"01JAN1999,1200,04JAN1999,1200",20SEC,10MIN,1,...,1,NaN,BaldEagleCr,None,12,C:\GH\ras-commander\examples\example_projects\...,11,C:\GH\ras-commander\examples\example_projects\...,C:\GH\ras-commander\examples\example_projects\...,Unsteady
5,03,13,09,Single 2D Area - Internal Dam Structure,5.04,Single 2D,"01JAN1999,1200,04JAN1999,1200",30SEC,10MIN,1,...,1,NaN,BaldEagleCr,None,09,C:\GH\ras-commander\examples\example_projects\...,13,C:\GH\ras-commander\examples\example_projects\...,C:\GH\ras-commander\examples\example_projects\...,Unsteady
6,04,01,13,SA to 2D Area Conn - 2D Levee Structure,5.00,2D Levee Struc,"01JAN1999,1200,04JAN1999,1200",20SEC,5MIN,1,...,1,NaN,BaldEagleCr,None,13,C:\GH\ras-commander\examples\example_projects\...,01,C:\GH\ras-commander\examples\example_projects\...,C:\GH\ras-commander\examples\example_projects\...,Unsteady
7,02,01,01,SA to Detailed 2D Breach,5.10,SA-2D Det Brch,"01JAN1999,1200,04JAN1999,1200",10SEC,5MIN,1,...,1,Pardiso (Direct),BaldEagleCr,None,01,C:\GH\ras-commander\examples\example_projects\...,01,C:\GH\ras-commander\examples\example_projects\...,C:\GH\ras-commander\examples\example_projects\...,Unsteady
8,01,01,01,SA to Detailed 2D Breach FEQ,5.03,SA-2D Det FEQ,"01JAN1999,1200,04JAN1999,1200",5SEC,5MIN,1,...,1,NaN,BaldEagleCr,None,01,C:\GH\ras-commander\examples\example_projects\...,01,C:\GH\ras-commander\examples\example_projects\...,C:\GH\ras-commander\examples\example_projects\...,Unsteady
9,05,02,03,Single 2D area with Bridges FEQ,5.10,Single 2D Bridges FEQ,"01JAN1999,1200,04JAN1999,1200",5SEC,10MIN,-1,...,1,PARDISO (Direct),BaldEagleCr,None,03,C:\GH\ras-commander\examples\example_projects\...,02,C:\GH\ras-commander\examples\example_projects\...,C:\GH\ras-commander\examples\example_projects\...,Unsteady


In [8]:
# Export Plan Numbers to List and Print
plan_numbers = ras.plan_df['plan_number'].tolist()
print(plan_numbers)


['13', '15', '17', '18', '19', '03', '04', '02', '01', '05', '06']


In [ ]:
# Define run_simulation function
import time
from ras_commander import RasGeo
from ras_commander.results import ResultsSummary

def run_simulation(version, plan_number):
    # Initialize project for the specific version
    ras_project = init_ras_project(project_path, str(version))
    
    # Clear geometry preprocessor files for the plan
    plan_path = RasPlan.get_plan_path(plan_number, ras_object=ras_project)
    RasGeo.clear_geompre_files(plan_path, ras_object=ras_project)
    
    # Set the number of cores to 4
    RasPlan.set_num_cores(plan_number, "4", ras_object=ras_project)
    
    # Update plan run flags – setting "Run HTab" flag to 1 to force geometry preprocessing
    RasPlan.update_run_flags(plan_number, {"Run HTab": 1}, ras_object=ras_project)
    
    # Compute the plan
    start_time = time.time()
    success = RasCmdr.compute_plan(plan_number, ras_object=ras_project)
    total_time = time.time() - start_time
    
    if success:
        # Get the HDF file path for the plan results
        hdf_path = RasPlan.get_results_path(plan_number, ras_object=ras_project)
        
        # Build plan metadata for summarize_plan
        plan_meta = {
            'plan_number': plan_number,
            'plan_title': f'Version {version}',
            'flow_type': 'Unsteady',
            'Program Version': version
        }
        
        # Use ResultsSummary.summarize_plan() which has fallback logic for pre-6.4 versions
        summary = ResultsSummary.summarize_plan(hdf_path, plan_meta)
        
        # Extract runtime values (works for all versions via fallback)
        if summary['runtime_complete_process_hours'] is not None:
            # Get preprocessor time - check for geometry-specific field first
            preprocessor_time = summary.get('runtime_geometry_hours')
            if preprocessor_time is None:
                # Fallback: estimate from complete process time - unsteady compute time
                preprocessor_time = (summary.get('runtime_complete_process_hours', 0) - 
                                   summary.get('runtime_unsteady_compute_hours', 0))
            
            unsteady_compute_time = summary.get('runtime_unsteady_compute_hours', 0)
            volume_error = summary.get('vol_error_percent')
            runtime_source = summary.get('runtime_source', 'unknown')
            
            # Print the extracted data
            print(f"\nExtracted Data for Plan {plan_number} in Version {version}:")
            print(f"Runtime Source: {runtime_source}")
            print(f"Preprocessor Time: {preprocessor_time:.3f} hr")
            print(f"Unsteady Compute Time: {unsteady_compute_time:.3f} hr") 
            print(f"Volume Error: {volume_error:.3f}%" if volume_error is not None else "Volume Error: None")
            print(f"Total Time: {total_time/3600:.3f} hr\n")
            
            return {
                'Version': version,
                'Plan': plan_number,
                'Preprocessor Time (hr)': preprocessor_time,
                'Unsteady Compute Time (hr)': unsteady_compute_time,
                'Volume Error (%)': volume_error,
                'Runtime Source': runtime_source,
                'Total Time (hr)': total_time / 3600  # convert seconds to hours
            }
        else:
            print(f"\n⚠️ Warning: No runtime data available for Version {version}, Plan {plan_number}")
            return None
    else:
        return None

In [10]:
# Select the plan number you want to run across all versions
plan_number = PLAN  # Use PLAN from parameters


In [11]:
# Run simulations for all versions with plan_number defined by user
results = []
for version in versions:
    print(f"Running simulation for Version {version}, Plan {plan_number}")
    result = run_simulation(version, plan_number) 
    if result is not None:  # Check if result is not None
        results.append(result)
        print(f"Completed: Version {version}, Plan {plan_number}")
    else:
        print(f"Failed: Version {version}, Plan {plan_number}")

# Create DataFrame from results
df = pd.DataFrame(results)

# Save initial results to CSV
df.to_csv('save_initial_results.csv', index=False)

print("Initial results saved to 'save_initial_results.csv'")


2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.g06
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 SA/2D connections in BaldEagleDamBrk.g06
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.g08
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 SA/2D connections in BaldEagleDamBrk.g08
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.g10
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 SA/2D connections in BaldEagleDamBrk.g10
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.g11
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 SA/2D connections in BaldEagleDamBrk.g11
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.

Running simulation for Version 6.6, Plan 02


2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.g09
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 SA/2D connections in BaldEagleDamBrk.g09
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.g13
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 SA/2D connections in BaldEagleDamBrk.g13
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.g01
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 SA/2D connections in BaldEagleDamBrk.g01
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.g03
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 SA/2D connections in BaldEagleDamBrk.g03
2026-01-19 16:36:57 - ras_commander.geom.GeomLateral - INFO - Found 0 lateral structures in BaldEagleDamBrk.


Extracted Data for Plan 02 in Version 6.6:
Preprocessor Time: 0.000 hr
Unsteady Compute Time: 0.367 hr
Volume Error: 0.007%
Total Time: 0.378 hr

Completed: Version 6.6, Plan 02
Running simulation for Version 6.5, Plan 02


2026-01-19 16:59:37 - ras_commander.hdf.HdfResultsPlan - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_701\BaldEagleDamBrk.p02.hdf
2026-01-19 16:59:37 - ras_commander.hdf.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_701\BaldEagleDamBrk.p02.hdf
2026-01-19 16:59:37 - ras_commander.hdf.HdfResultsPlan - INFO - Reading computation messages from HDF: BaldEagleDamBrk.p02.hdf
2026-01-19 16:59:37 - ras_commander.hdf.HdfResultsPlan - INFO - Successfully extracted 28604 characters from HDF
2026-01-19 16:59:37 - ras_commander.hdf.HdfResultsPlan - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_701\BaldEagleDamBrk.p02.hdf
2026-01-19 16:59:37 - ras_commander.hdf.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_701\BaldEagleDamBrk.p02.hdf



Extracted Data for Plan 02 in Version 6.5:
Preprocessor Time: 0.000 hr
Unsteady Compute Time: 0.368 hr
Volume Error: 0.005%
Total Time: 0.379 hr

Completed: Version 6.5, Plan 02
Running simulation for Version 6.4.1, Plan 02


2026-01-19 17:22:21 - ras_commander.hdf.HdfResultsPlan - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_701\BaldEagleDamBrk.p02.hdf
2026-01-19 17:22:21 - ras_commander.hdf.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_701\BaldEagleDamBrk.p02.hdf
2026-01-19 17:22:21 - ras_commander.hdf.HdfResultsPlan - INFO - Reading computation messages from HDF: BaldEagleDamBrk.p02.hdf
2026-01-19 17:22:21 - ras_commander.hdf.HdfResultsPlan - INFO - Successfully extracted 1568 characters from HDF
2026-01-19 17:22:21 - ras_commander.hdf.HdfResultsPlan - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_701\BaldEagleDamBrk.p02.hdf
2026-01-19 17:22:21 - ras_commander.hdf.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_701\BaldEagleDamBrk.p02.hdf
2


Extracted Data for Plan 02 in Version 6.4.1:
Preprocessor Time: 0.000 hr
Unsteady Compute Time: 0.325 hr
Volume Error: 0.006%
Total Time: 0.335 hr

Completed: Version 6.4.1, Plan 02
Running simulation for Version 6.3.1, Plan 02


2026-01-19 17:42:29 - ras_commander.hdf.HdfResultsPlan - INFO - Reading computation messages from HDF: BaldEagleDamBrk.p02.hdf
2026-01-19 17:42:29 - ras_commander.hdf.HdfResultsPlan - INFO - Successfully extracted 1564 characters from HDF
2026-01-19 17:42:29 - ras_commander.hdf.HdfResultsPlan - INFO - Using existing Path object HDF file: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_701\BaldEagleDamBrk.p02.hdf
2026-01-19 17:42:29 - ras_commander.hdf.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_701\BaldEagleDamBrk.p02.hdf
2026-01-19 17:42:29 - ras_commander.hdf.HdfResultsPlan - INFO - Extracting Plan Information from: BaldEagleDamBrk.p02.hdf
2026-01-19 17:42:29 - ras_commander.hdf.HdfResultsPlan - INFO - Plan Name: SA to Detailed 2D Breach
2026-01-19 17:42:29 - ras_commander.hdf.HdfResultsPlan - INFO - Simulation Duration (hours): 72.0
2026-01-19 17:42:29 - ras_commander.hdf.HdfResultsPlan - INFO

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Create line graphs
plt.figure(figsize=(12, 6))

# Unsteady Runtime vs Version
plt.subplot(1, 2, 1)
# Convert Version to categorical type to handle string versions properly
plt.plot(pd.Categorical(df['Version']), df['Unsteady Compute Time (hr)'], marker='o')
plt.title(f'Unsteady Runtime vs HEC-RAS Version (Plan {plan_number})')
plt.xlabel('HEC-RAS Version')
plt.ylabel('Unsteady Runtime (hours)')
plt.grid(True, linestyle='--', alpha=0.7)

# Volume Error vs Version
plt.subplot(1, 2, 2)
plt.plot(pd.Categorical(df['Version']), df['Volume Error (%)'], marker='o')
plt.title(f'Volume Error vs HEC-RAS Version (Plan {plan_number})')
plt.xlabel('HEC-RAS Version')
plt.ylabel('Volume Error (%)')
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
# Benchmark all plans with 2 cores in HEC-RAS 6.6
results_2core = []

# Loop through each plan number
for plan in plan_numbers:
    print(f"Running simulation for Version {ALL_PLANS_VERSION}, Plan {plan} with {ALL_PLANS_CORES} cores")
    
    # Initialize project for specified version
    ras_project = init_ras_project(project_path, ALL_PLANS_VERSION)
    
    # Clear geometry preprocessor files
    plan_path = RasPlan.get_plan_path(plan, ras_object=ras_project)
    RasGeo.clear_geompre_files(plan_path, ras_object=ras_project)
    
    # Set number of cores
    RasPlan.set_num_cores(plan, str(ALL_PLANS_CORES), ras_object=ras_project)
    
    # Update plan run flags
    RasPlan.update_run_flags(plan, {"Run HTab": 1}, ras_object=ras_project)
    
    # Compute the plan
    start_time = time.time()
    success = RasCmdr.compute_plan(plan, ras_object=ras_project)
    total_time = time.time() - start_time
    
    if success:
        # Get HDF file path
        hdf_path = RasPlan.get_results_path(plan, ras_object=ras_project)
        
        # Build plan metadata for summarize_plan
        plan_meta = {
            'plan_number': plan,
            'plan_title': ras.plan_df.loc[ras.plan_df['plan_number'] == plan, 'Plan Title'].values[0],
            'flow_type': 'Unsteady',
            'Program Version': ALL_PLANS_VERSION
        }
        
        # Use ResultsSummary.summarize_plan() which has fallback logic for pre-6.4 versions
        summary = ResultsSummary.summarize_plan(hdf_path, plan_meta)
        
        # Extract runtime values (works for all versions via fallback)
        if summary['runtime_complete_process_hours'] is not None:
            # Get preprocessor time - check for geometry-specific field first
            preprocessor_time = summary.get('runtime_geometry_hours')
            if preprocessor_time is None:
                # Fallback: estimate from complete process time - unsteady compute time
                preprocessor_time = (summary.get('runtime_complete_process_hours', 0) - 
                                   summary.get('runtime_unsteady_compute_hours', 0))
            
            unsteady_compute_time = summary.get('runtime_unsteady_compute_hours', 0)
            volume_error = summary.get('vol_error_percent')
            
            result = {
                'Plan': plan,
                'Preprocessor Time (hr)': preprocessor_time,
                'Unsteady Compute Time (hr)': unsteady_compute_time,
                'Volume Error (%)': volume_error,
                'Total Time (hr)': total_time / 3600
            }
            results_2core.append(result)
            print(f"Completed: Plan {plan} with {ALL_PLANS_CORES} cores")
        else:
            print(f"⚠️ Warning: No runtime data available for Plan {plan}")
    else:
        print(f"Failed: Plan {plan} with {ALL_PLANS_CORES} cores")

# Convert results to DataFrame
df_2core = pd.DataFrame(results_2core)

# Get plan titles from ras.plan_df and merge with results
plan_titles = pd.DataFrame({
    'Plan': ras.plan_df['plan_number'].str.zfill(2),
    'Short Identifier': ras.plan_df['Short Identifier']
})
df_2core['Plan'] = df_2core['Plan'].astype(str).str.zfill(2)
df_2core = df_2core.merge(plan_titles, on='Plan', how='left')

# Create visualization
plt.figure(figsize=(15, 10))

# Plot 1: Unsteady Runtime
plt.subplot(2, 2, 1)
bars = plt.bar(range(len(df_2core)), df_2core['Unsteady Compute Time (hr)'], color='blue', alpha=0.7)
plt.title(f'Unsteady Runtime by Plan ({ALL_PLANS_CORES} Cores)', fontsize=12)
plt.ylabel('Unsteady Runtime (hours)', fontsize=10)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(range(len(df_2core)), [f"Plan {plan}\n{title}" for plan, title in zip(df_2core['Plan'], df_2core['Short Identifier'])], rotation=45, ha='right')

# Plot 2: Volume Error
plt.subplot(2, 2, 2)
plt.bar(range(len(df_2core)), df_2core['Volume Error (%)'], color='red', alpha=0.7)
plt.title(f'Volume Error by Plan ({ALL_PLANS_CORES} Cores)', fontsize=12)
plt.ylabel('Volume Error (%)', fontsize=10)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(range(len(df_2core)), [f"Plan {plan}\n{title}" for plan, title in zip(df_2core['Plan'], df_2core['Short Identifier'])], rotation=45, ha='right')

# Plot 3: Preprocessor Time
plt.subplot(2, 2, 3)
plt.bar(range(len(df_2core)), df_2core['Preprocessor Time (hr)'], color='green', alpha=0.7)
plt.title(f'Preprocessor Time by Plan ({ALL_PLANS_CORES} Cores)', fontsize=12)
plt.ylabel('Preprocessor Time (hours)', fontsize=10)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(range(len(df_2core)), [f"Plan {plan}\n{title}" for plan, title in zip(df_2core['Plan'], df_2core['Short Identifier'])], rotation=45, ha='right')

# Plot 4: Total Runtime
plt.subplot(2, 2, 4)
plt.bar(range(len(df_2core)), df_2core['Total Time (hr)'], color='purple', alpha=0.7)
plt.title(f'Total Runtime by Plan ({ALL_PLANS_CORES} Cores)', fontsize=12)
plt.ylabel('Total Runtime (hours)', fontsize=10)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(range(len(df_2core)), [f"Plan {plan}\n{title}" for plan, title in zip(df_2core['Plan'], df_2core['Short Identifier'])], rotation=45, ha='right')

plt.tight_layout(pad=3.0)
plt.suptitle(f'Plan Performance Comparison ({ALL_PLANS_CORES} Cores)', fontsize=14, y=1.02)
plt.show()

# Save results to CSV
df_2core.to_csv('hecras_plan_comparison_2core.csv', index=False)
print("Results saved to 'hecras_plan_comparison_2core.csv'")

# Display summary statistics
print("\nSummary Statistics:")
print(df_2core.describe())